In [ ]:
!git clone https://github.com/Srija616/DeepLearning_Assignment_2.git

In [ ]:
!pip install wandb
import wandb

In [1]:
import os
import wandb
import random
import numpy as np

import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset, random_split, Subset
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
torch.manual_seed(42)

In [2]:
path = os.path.dirname(os.path.realpath('__file__'))
train_path = os.path.join(path, 'inaturalist_12K/train/')
test_path = os.path.join(path, 'inaturalist_12K/val/')
print (train_path, test_path)

/home/vg/ttsteam/repos/srija/inaturalist_12K/train/ /home/vg/ttsteam/repos/srija/inaturalist_12K/val/


In [3]:
def preprocess_data(train_path, test_path, data_aug=False):
    if data_aug == True:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                            T.RandomHorizontalFlip(p=0.5),
                            T.RandomVerticalFlip(p=0.5),
                            T.RandomRotation(degrees=45),
                            T.ToTensor(),
                            T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
                                
                ])
    else:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                    T.ToTensor(),
                    T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])])

    dataset = ImageFolder(train_path, transform=transforms)
    test_dataset = ImageFolder(test_path, transform=transforms)

    train_size = int(0.8*len(dataset))
    val_size = (len(dataset) - train_size)

    class_labels = [label for _, label in dataset]
    target = np.array(class_labels)

    from sklearn.model_selection import StratifiedShuffleSplit
    stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    for train_idx, val_idx in stratified_splitter.split(np.zeros(len(target)), target):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

    return train_dataset, val_dataset, test_dataset

In [5]:
import sys
sys.path.append("/home/vg/ttsteam/repos/srija/DeepLearning_Assignment_2/models")
from model_partb import FinetuneImgModel
import pytorch_lightning as pl

In [6]:
config = {
    'data_aug':True
}

In [ ]:
def finetuneimgmodel():
  train_dataset, val_dataset, test_dataset = preprocess_data(train_path, test_path, config["data_aug"])
  model = FinetuneImgModel(
      train_dataset = train_dataset,
      test_dataset = test_dataset,
      val_dataset = val_dataset 
  )
